In [1]:
import math
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('../data/scootertestdata.csv')

In [4]:
data.head

<bound method NDFrame.head of        company                               bike_id   timestamp        lat  \
0         lime  211958cd-b9bb-40c7-aaee-59445797ae4a  1626385005  47.599300   
1         lime  e780834c-5afb-4c87-ab96-19daea355776  1626385005  47.578400   
2         lime  dda23ebf-6120-46bb-96bf-2caad4e5b9a6  1626385005  47.574700   
3         lime  670ab889-198c-4738-a6e9-f1c8b68880ea  1626385005  47.599600   
4         lime  a65df0d8-dddc-41a1-bc74-c8acb4958b04  1626385005  47.605600   
...        ...                                   ...         ...        ...   
182687  wheels  8d9d91d3-e775-4d55-9967-7a7e55e1435f  1626387967  47.612385   
182688  wheels  025fadb7-3967-43c3-919c-34999a4a5358  1626387967  47.618027   
182689  wheels  04b2352a-40f2-4fc5-803c-db853640642f  1626387967  47.622501   
182690  wheels  ab943b74-dbab-41f5-8952-e1ee2a58223e  1626387967  47.611641   
182691  wheels  631115f3-584c-44b5-beee-a432ad682348  1626387967  47.616352   

               lon  i

In [22]:
data[data['company']=='lime']

,company,bike_id,timestamp,lat,lon,is_reserved,is_disabled,vehicle_type,vehicle_type_id
0,lime,211958cd-b9bb-40c7-aaee-59445797ae4a,1626385005,47.5993,-122.3299,0,0,bike,NaN
1,lime,e780834c-5afb-4c87-ab96-19daea355776,1626385005,47.5784,-122.3301,0,0,bike,NaN
2,lime,dda23ebf-6120-46bb-96bf-2caad4e5b9a6,1626385005,47.5747,-122.3302,0,0,scooter,NaN
3,lime,670ab889-198c-4738-a6e9-f1c8b68880ea,1626385005,47.5996,-122.3303,0,0,bike,NaN
4,lime,a65df0d8-dddc-41a1-bc74-c8acb4958b04,1626385005,47.6056,-122.3309,0,0,bike,NaN
...,...,...,...,...,...,...,...,...,...
166022,lime,5cb1341e-8f03-449f-b505-38c8d3d6ea99,1626387795,47.5784,-122.4132,0,0,scooter,NaN
166023,lime,70f3a3a5-d7f1-4922-b674-c77deda1a73e,1626387795,47.6545,-122.4132,0,0,scooter,NaN
166024,lime,7963cb6e-27c0-49db-9362-fa3e7ee73964,1626387795,47.5725,-122.4135,0,0,scooter,NaN
166025,lime,819d51fb-2202-43be-9047-227d2a10b3d8,1626387795,47.6485,-122.4142,0,0,bike,NaN


In [16]:
data[data['bike_id']=='025fadb7-3967-43c3-919c-34999a4a5358']

,company,bike_id,timestamp,lat,lon,is_reserved,is_disabled,vehicle_type,vehicle_type_id
3925,wheels,025fadb7-3967-43c3-919c-34999a4a5358,1626385361,47.618027,-122.319801,0,0,NaN,NaN
7850,wheels,025fadb7-3967-43c3-919c-34999a4a5358,1626385366,47.618027,-122.319801,0,0,NaN,NaN
10614,wheels,025fadb7-3967-43c3-919c-34999a4a5358,1626385402,47.618027,-122.319801,0,0,NaN,NaN
13378,wheels,025fadb7-3967-43c3-919c-34999a4a5358,1626385441,47.618027,-122.319801,0,0,NaN,NaN
16140,wheels,025fadb7-3967-43c3-919c-34999a4a5358,1626385475,47.618027,-122.319801,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
171574,wheels,025fadb7-3967-43c3-919c-34999a4a5358,1626387831,47.618027,-122.319801,0,0,NaN,NaN
174354,wheels,025fadb7-3967-43c3-919c-34999a4a5358,1626387865,47.618027,-122.319801,0,0,NaN,NaN
177133,wheels,025fadb7-3967-43c3-919c-34999a4a5358,1626387899,47.618027,-122.319801,0,0,NaN,NaN
179911,wheels,025fadb7-3967-43c3-919c-34999a4a5358,1626387933,47.618027,-122.319801,0,0,NaN,NaN
